In [1]:
import numpy as np
import numpy.random as rnd
import collections
from enum import Enum

In [2]:


def dice():
    vals = ['skull', 'coin', 'diamond', 'parrot', 'monkey', 'sword']
    return vals[rnd.choice(5)]

def card():
    return naive_card()

def naive_card():
    all_cards = []
    return all_cards[rnd.choice(len(all_cards) - 1)]

In [3]:
class Game():
    def __init__(self, players = 2):        
        self.players = players
        self.scores = [0] * players
        self.turn = 0
        self.dice_count = 8
    
    def to_string(self):
        return '# scores: %s' % (self.scores)

    

In [4]:
class Card_Values(Enum):
        SORCERESS = 0
        PARROTS_AND_MONKEYS = 1
        CAPTAIN = 2
        SEA_BATTLE = 3
        COIN = 4
        DIAMOND = 5
        SKULL = 6
        TWO_SKULLS = 6
        CHEST = 7

In [7]:
class Dice_Values(Enum):
    SKULL = 0
    COIN = 1
    DIAMOND = 2
    PARROT = 3
    MONKEY = 4
    SWORD = 5

class Dices():
    def __init__(self, count = 8):
        self.dice_count = count
        self.dice_values = [None] * count
        
        self._all_dice_vals = [d for d in Dice_Values]
    
    def get_current(self):
        #return a copy
        return [x for x in self.dice_values]
    
    def get_current_as_str(self):
        #return a copy
        return [x.name for x in self.dice_values]
    
    def roll(self, indexes = None):
        if (indexes is None):
            indexes = range(self.dice_count)
        
        for idx in indexes:
            v = self._single_roll()
            self.dice_values[idx] = v
            
        return self
    
    def _single_roll(self):        
        return self._all_dice_vals[rnd.choice(6)]    
    
    def to_string(self):
        return '%s' % (str([x.name for x in self.get_current()]))



#test partial roll
d = Dices()

idx = [0,1,2,4]
anti_idx = [x for x in range(8) if x not in idx]
for i in range(10):
    vals = d.roll().get_current()
    partial_roll = d.roll(idx).get_current()
    assert all([vals[i] == partial_roll[i] for i in anti_idx ])



In [8]:
d.roll().get_current_as_str()

['MONKEY', 'MONKEY', 'MONKEY', 'SWORD', 'SKULL', 'SWORD', 'SWORD', 'DIAMOND']

In [9]:
class Scoring():
    def __init__(self, dice_values, card = None, dice_count = 8):
        
        self.dice_values = self._convert_dices_to_enum(dice_values)
        self.card = self._convert_card_to_enum(card)
        self._dice_count = dice_count
        
        self._scores_by_count = {3: 100, 4: 200, 5: 500, 6: 1000, 7: 2000, 8: 4000}
        self._full_chest = 500
        self._valuables = {Dice_Values.COIN: 100, Dice_Values.DIAMOND: 100}
        self._dice_used = set() #used for full-chest scoring
    
    def _convert_dices_to_enum(self, dice_values):
        return [x if x in Dice_Values else Dice_Values[x.upper()] for x in dice_values]
    
    def _convert_card_to_enum(self, card):
        if (card is None):
            return None
        
        return card if card in Card_Values else Card_Values[card.upper()]
        
    def _add_card_to_dice(self):
        values = self.dice_values
        
        if (self.card == Card_Values.COIN):
            values += Dice_Values.COIN
        elif (self.card ==  Card_Values.DIAMOND):
            values += Dice_Values.DIAMOND
        elif (self.card ==  Card_Values.SKULL):
            values += Dice_Values.SKULL
        elif (self.card ==  Card_Values.TWO_SKULLS):
            values += [Dice_Values.SKULL] * 2
            
        return values
    
    def is_disqualified(self):
        return len([x for x in self.dice_values if x == Dice_Values.SKULL]) >= 3
        
    def score(self):
        self.dice_values = self._add_card_to_dice()
        
        if (self.is_disqualified()):
            return 0
            
        score = self._score_by_groups() \
                + self._score_by_valuables() \
                + self._score_by_full_chest() \
                + self._score_by_sea_battle()
                
        
        coef = 2. if (self.card == Card_Values.CAPTAIN) else 1.
        
        return coef * score
    
    def _score_by_valuables(self):
        dice_used = {i for i,v in enumerate(self.dice_values) if v in self._valuables}
        self._dice_used |= dice_used
        
        return sum([self._valuables.get(x, 0) for x in self.dice_values])
    
    def _score_by_groups(self):
        
        if (self.card == Card_Values.PARROTS_AND_MONKEYS):
            self.dice_values = [Dice_Values.MONKEY if x == Dice_Values.PARROT else x for x in self.dice_values]
                    
        counts = collections.Counter(self.dice_values)
        groups_used = {k: v for k,v in counts.items() if v >= 3}
        dice_used = {i for i,v in enumerate(self.dice_values) if v in groups_used}
        self._dice_used |= dice_used        
        
        return sum([self._scores_by_count.get(v, 0) for k,v in counts.items()])
    
    #must be called after other scoring methods
    def _score_by_full_chest(self):
        if (len(self._dice_used) >= self._dice_count):
            return self._full_chest
        
        return 0
    
    #should be here? not sure.
    def _score_by_sea_battle(self):
        #ToDo
        return 0
        



#nothing
assert Scoring(['parrot', 'parrot', 'monkey', 'monkey']).score() == 0.0
#parrots and monkeys
assert Scoring(['parrot', 'parrot', 'monkey', 'monkey'], 'parrots_and_monkeys').score() == 200.0
#captain
assert Scoring(['sword', 'sword', 'sword', 'sword'], 'captain').score() == 400.0
#coin, diamond
assert Scoring(['sword', 'sword', 'sword', 'sword', 'diamond', 'coin'], 'captain').score() == 800.0
#skulls
assert Scoring(['sword', 'sword', 'skull', 'sword', 'skull', 'coin', 'skull'], 'captain').score() == 0.0
#full chest control
assert Scoring(['sword', 'sword', 'sword', 'sword', 'coin', 'coin', 'diamond', 'monkey'], None).score() == 500.0
#Full chest
assert Scoring(['sword', 'sword', 'sword', 'sword', 'coin', 'coin', 'diamond', 'diamond'], None).score() == 1100.0

In [10]:
g = Game()
g.to_string()

'# scores: [0, 0]'